In [1]:
import ray
from ray import tune


In [47]:
tuner = tune.Tuner.restore(
    path='HPO/tiny-imagenet/rs/vgg16aw',
    
)


/root/notebooks/nfs/work/larry.lai/AWPooling/lib/python3.8/site-packages/ray/tune/tuner.py:230: UserWarning: Passing in the experiment's `trainable` will be a required argument to `Tuner.restore` starting from version 2.5. Please specify the trainable to avoid this warning.
  warnings.warn(warning_message)
2023-05-14 16:27:01,365	INFO experiment_analysis.py:789 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.


In [49]:
tune.Tuner.restore?

Signature:
tune.Tuner.restore(
    path: str,
    trainable: Union[str, Callable, Type[ray.tune.trainable.trainable.Trainable], ForwardRef('BaseTrainer'), NoneType] = None,
    resume_unfinished: bool = True,
    resume_errored: bool = False,
    restart_errored: bool = False,
    overwrite_trainable: Union[str, Callable, Type[ray.tune.trainable.trainable.Trainable], ForwardRef('BaseTrainer'), NoneType] = None,
) -> 'Tuner'
Docstring:
Restores Tuner after a previously failed run.

All trials from the existing run will be added to the result table. The
argument flags control how existing but unfinished or errored trials are
resumed.

Finished trials are always added to the overview table. They will not be
resumed.

Unfinished trials can be controlled with the ``resume_unfinished`` flag.
If ``True`` (default), they will be continued. If ``False``, they will
be added as terminated trials (even if they were only created and never
trained).

Errored trials can be controlled with the ``resum

In [4]:
search_space = {
    'a': tune.uniform(0, 20),
    'b': tune.uniform(0, 20),
}

tune_config = tune.TuneConfig(
    num_samples=10,
)

run_config = ray.air.config.RunConfig(
    name='test',
    local_dir='HPO',
)

tuner = tune.Tuner(
    F,
    tune_config=tune_config,
    run_config=run_config,
    param_space=search_space,
)

tuner.fit()

2023-05-14 15:18:50,188	INFO worker.py:1553 -- Started a local Ray instance.


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,score,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
F_9441f_00000,2023-05-14_15-19-08,True,,b43e794fb767465e83741014db4d219c,"0_a=0.8756,b=3.5933",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11578,-1.84203,0.000451326,0.000451326,0.000451326,1684048748,0,,1,9441f_00000,0.00913882
F_9441f_00001,2023-05-14_15-19-14,True,,8c737d81a92e4824af32854f4f9e7791,"1_a=8.7612,b=13.1859",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11831,4.33644,0.000419855,0.000419855,0.000419855,1684048754,0,,1,9441f_00001,0.00562906
F_9441f_00002,2023-05-14_15-19-14,True,,17be0377886b4ac6a3db556cafa3f478,"2_a=12.4049,b=1.6846",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11833,23.1253,0.000422001,0.000422001,0.000422001,1684048754,0,,1,9441f_00002,0.00616026
F_9441f_00003,2023-05-14_15-19-14,True,,11abdebfe79b4ce38512b20226219465,"3_a=0.2104,b=5.9463",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11835,-5.52552,0.000360489,0.000360489,0.000360489,1684048754,0,,1,9441f_00003,0.00653028
F_9441f_00004,2023-05-14_15-19-14,True,,393f9a1219604fd3a8f5752eb62634ac,"4_a=11.6121,b=17.1240",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11837,6.10014,0.000399351,0.000399351,0.000399351,1684048754,0,,1,9441f_00004,0.00719094
F_9441f_00005,2023-05-14_15-19-14,True,,1ee01ac5a60b401182c2103add9adeaa,"5_a=13.4373,b=1.9873",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11839,24.8873,0.00037694,0.00037694,0.00037694,1684048754,0,,1,9441f_00005,0.0090394
F_9441f_00006,2023-05-14_15-19-14,True,,1eaea5b2fdc64c6b9d9303f77e7bca55,"6_a=8.4721,b=0.1255",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11841,16.8187,0.000276804,0.000276804,0.000276804,1684048754,0,,1,9441f_00006,0.00617766
F_9441f_00007,2023-05-14_15-19-14,True,,b8bc592bf73648fd929ca69291ae91f7,"7_a=12.5954,b=15.1513",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11843,10.0395,0.000374794,0.000374794,0.000374794,1684048754,0,,1,9441f_00007,0.00601983
F_9441f_00008,2023-05-14_15-19-14,True,,08f3a707f6964d99b339160b3a5d6466,"8_a=11.3128,b=6.4413",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11845,16.1843,0.00037694,0.00037694,0.00037694,1684048754,0,,1,9441f_00008,0.006495
F_9441f_00009,2023-05-14_15-19-14,True,,692f5029e4bd4ed2abc1716236eed520,"9_a=14.0062,b=18.5766",forge-2wr458lt-310832011-7748c95cd4-8hqfq,1,10.233.83.2,11853,9.43568,0.000372648,0.000372648,0.000372648,1684048754,0,,1,9441f_00009,0.0059526


2023-05-14 15:19:14,599	INFO tune.py:798 -- Total run time: 22.28 seconds (10.49 seconds for the tuning loop).


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
from glob import glob
import numpy as np
import os

csv_path = glob('HPO/tiny-imagenet/scratch/vgg19aw/*.csv')
save_path = 'HPO/performance'
analysis = []

for path in csv_path: # read expermiment record
    model_name = path.split('/')[-2]
    csv = pd.read_csv(path)
    
    for index, row in csv.iterrows():
        acc = row['accs']
        tmp = {'acc': acc, 'pool': []}
        for tem in row['tems'].split(','):
            if tem == '':
                break
            
            name, t = tem.split('=')
            t = float(t)
            
            if t <= 0.1:
                tmp['pool'].append('max')
            elif t >= 5:
                tmp['pool'].append('avg')
            else:
                tmp['pool'].append('sw')
        analysis.append(tmp)
        

    
#     # plot result
#     for k, v in analysis.items():
        
#         plt.title(f'Model performace {k}')
#         plt.xticks(np.arange(0, 11, 1))
#         plt.xlabel("Temperature")
#         plt.ylabel("Accuracy")
#         plt.scatter(*zip(*v['max']), c='g', marker=6, label='max')
#         plt.scatter(*zip(*v['avg']), c='b', marker=6, label='avg')
#         plt.scatter(*zip(*v['sw']), c='r', marker=6, label='sw')
#         plt.legend(loc=1)
#         plt.savefig(os.path.join(save_path, f'{model_name}_{k}.png'))
#         plt.clf()
                  
# csv = pd.read_csv(csv_path[0])
# for index, row in csv.iterrows():
#     tem = row['tems'].split(',')
#     t0 = float(tem[0][3:])
#     t4 = float(tem[4][3:])
#     acc = row['accs']
#     if t4 < 1:
#         analysis['max'].append((t4, acc))
#     elif t4 > 5:
#         analysis['avg'].append((t4, acc))
#     else:
#         analysis['sw'].append((t4, acc))
        
# plt.title(f'Independent analysis {t}')
# plt.xticks(np.arange(0, 11, 1))
# plt.scatter(*zip(*analysis['max']), c='g')
# plt.scatter(*zip(*analysis['avg']), c='b')
# plt.scatter(*zip(*analysis['sw']), c='r')
# plt.savefig(os.path.join(save_path, f'{t}_acc.png'))
# plt.show()


{'acc': [(0.561199963092804, 's'), (0.5543999671936035, 'a'), (0.5496000051498413, 's'), (0.5496000051498413, 'a')], 'pool': 'assa'}
{'acc': [(0.5601999759674072, 'a'), (0.5511999726295471, 's'), (0.5496000051498413, 's'), (0.5425999760627747, 's')], 'pool': 'asss'}
{'acc': [(0.5601999759674072, 's'), (0.5573999881744385, 'a'), (0.5532000064849854, 's'), (0.5529999732971191, 'a'), (0.5509999990463257, 's'), (0.5501999855041504, 's'), (0.5501999855041504, 'a'), (0.5478000044822693, 's'), (0.5471999645233154, 's'), (0.5453999638557434, 's'), (0.5447999835014343, 's')], 'pool': 'aaaa'}
{'acc': [(0.5582000017166138, 's'), (0.5569999814033508, 's'), (0.5507999658584595, 's'), (0.5491999983787537, 'a'), (0.5478000044822693, 's'), (0.54339998960495, 's'), (0.5356000065803528, 's')], 'pool': 'aass'}
{'acc': [(0.5561999678611755, 'a'), (0.5559999942779541, 'a'), (0.5546000003814697, 'a'), (0.5507999658584595, 'a'), (0.5507999658584595, 'a'), (0.550599992275238, 's'), (0.5489999651908875, 's'), 

torch.Size([3])

RuntimeError: shape '[100, 200]' is invalid for input of size 6400

In [52]:
import torch
import torch.nn as nn
from models.awpooling import *
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.tensorboard import SummaryWriter

from models.vggaw import VGG11AW, VGG11AWT
import os

h = 1e-4
writer = SummaryWriter(log_dir=os.path.join('Gradient_analysis', f'delta={h}'))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
ds = ImageFolder(root='/home/larry/Datasets/tiny-imagenet-200/train', transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
]))
loader = torch.utils.data.DataLoader(ds, shuffle=True, batch_size=128, num_workers=2)

In [42]:
class Net(nn.Module):
    def __init__(self, num_class=200):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
        )
        self.aw1 = AWPool2d_()

        # self.conv2 = nn.Sequential(
        #     nn.Conv2d(64, 128, kernel_size=3, padding=1),
        #     nn.ReLU(),
        #     nn.BatchNorm2d(128),
        #     nn.Conv2d(128, 128, kernel_size=3, padding=1),
        #     nn.ReLU(),
        #     nn.BatchNorm2d(128),
        # )
        # self.aw2 = AWPool2d_()
    
        self.globalavg = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(),
            # nn.Dropout(),
            nn.Linear(256, 256),
            nn.ReLU(),
            # nn.Dropout(),
            nn.Linear(256, num_class),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.aw1(x)
        x = self.globalavg(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    def set_t(self, t):
        self.aw1.t = t


In [43]:
model = Net()
params = [{'params': p, 'lr': 0.1} for n, p in model.named_parameters() if 'aw' not in n]
optimizer = torch.optim.SGD(params)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (aw1): AWPool2d_()
  (globalavg): AdaptiveAvgPool2d(output_size=1)
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=200, bias=True)
  )
)

In [44]:
for i, data in enumerate(loader):
    model.aw1.t.requires_grad = True
    images, label = data
    images = images.to(device)
    label = label.to(device)

    # Autograd
    logits = model(images)
    base_loss = criterion(logits, label)
    base_loss.backward()
    writer.add_scalar('Automatic gardient', model.aw1.t.grad, i)

    # caculate numercial difference
    with torch.no_grad():
        model.aw1.t.requires_grad = False
        origin_t = model.aw1.t.item()

        # forward
        model.aw1.t.copy_(torch.tensor(origin_t + h))
        logits = model(images)
        forward_loss = criterion(logits, label)

        # backward
        model.aw1.t.copy_(torch.tensor(origin_t - h))
        logits = model(images)
        backward_loss = criterion(logits, label)

        writer.add_scalar('Forward difference', (forward_loss - base_loss) / h, i)
        writer.add_scalar('Backward difference', (base_loss - backward_loss) / h, i)
        writer.add_scalar('Central difference', (forward_loss - backward_loss) / (2 * h), i)

    model.aw1.t.copy_(torch.tensor(origin_t))

    optimizer.step()
    optimizer.zero_grad()
    model.aw1.t.grad.zero_()
    

In [33]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [34]:
%tensorboard --logdir Gradient_analysis

In [5]:
logits = model(x)
loss = criterion(logits, y)
loss.backward()
print(model.aw1.t.grad)

tensor([0.0037], device='cuda:0')


In [ ]:
model.aw1.t.requires_grad = False
logits = model(x)
base = criterion(logits, y)

In [12]:
# compute forward loss
model.aw1.t.copy_(torch.tensor(origin_t + h))
print(model.aw1.t.item())
logits = model(x)
forward = criterion(logits, y)
model.aw1.t.copy_(torch.tensor(origin_t - h))
print(model.aw1.t.item())
logits = model(x)
backward = criterion(logits, y)

model.aw1.t.copy_(torch.tensor(origin_t))
print(f'base loss: {base.item()}\nforward loss: {forward.item()}\nbackward loss: {backward.item()}\n'\
      f'forward difference: {(forward - base) / h}\n'\
      f'backward difference: {(base - backward) / h}\n'\
      f'central difference: {(forward - backward) / (2 * h)}')


1.0019999742507935
0.9980000257492065
base loss: 5.295560836791992
forward loss: 5.295570373535156
backward loss: 5.29556131362915
forward difference: 0.004768371116369963
backward difference: -0.00023841856454964727
central difference: 0.0022649762686342


In [13]:
def compute_numerical(model, func, x, h=1e-4):
    logits = func(x)
    forward = func(x + h)
    backward = func(x - h)

    print(f'forward difference {(forward - logits) / h}\nbackward difference: {(logits - backward) / h}\ncentral difference: {(forward-backward)/ (2* h)}')

In [ ]:
def softmax_t(x, t=1):
    return np.exp(x / t) / np.sum(np.exp(x / t))

In [2]:
def softmax_pool(x, t):
    return x.dot(softmax_t(x, t))

In [134]:
a = torch.tensor(0.05)
b = a.item()
a.copy_(torch.tensor(3))
print(b, a)


0.05000000074505806 tensor(3.)
